In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb

from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission (1).csv
Saving testFeatures.csv to testFeatures (1).csv
Saving train.csv to train (1).csv


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("testFeatures.csv")

train["dataset"] = "train"
test["dataset"] = "test"
test["ürün fiyatı"] = None

df = pd.concat([train, test], ignore_index=True)


<ipython-input-18-4307ec0684a6>:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([train, test], ignore_index=True)


In [ ]:

# 🔁 Kategorik sütunlara id'yi de dahil et
cat_cols = ["id", "ürün", "ürün kategorisi", "ürün üretim yeri", "market", "şehir"]
for col in cat_cols:
    df[col] = LabelEncoder().fit_transform(df[col])


In [ ]:
df["tarih"] = pd.to_datetime(df["tarih"])
df["yıl"] = df["tarih"].dt.year
df["ay"] = df["tarih"].dt.month
df["gün"] = df["tarih"].dt.day
df.drop(columns=["tarih"], inplace=True)


In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = SimpleImputer(strategy="mean").fit_transform(df[numeric_cols])


In [ ]:
from sklearn.neighbors import LocalOutlierFactor

train_df = df[df["dataset"] == "train"].copy()
X_lof = train_df.drop(columns=["ürün fiyatı", "dataset"])
lof = LocalOutlierFactor(n_neighbors=20)
outliers = lof.fit_predict(X_lof)
train_df = train_df[outliers != -1]


In [ ]:
from sklearn.cluster import KMeans

df["ürün_grubu"] = KMeans(n_clusters=5, random_state=42).fit_predict(df[["ürün", "ürün besin değeri"]])


In [ ]:
df = df.drop(columns=["dataset"])

X_train = train_df.drop(columns=["ürün fiyatı"])
y_train = train_df["ürün fiyatı"]
X_test = df.iloc[len(train_df):].drop(columns=["ürün fiyatı"])


In [ ]:
X_train = X_train.select_dtypes(include=["number"])
X_test = X_test[X_train.columns]



In [ ]:
params = {
    "objective": "regression",
    "metric": "rmse",
    "boosting_type": "gbdt",
    "verbosity": -1,
    "learning_rate": 0.1,
    "num_leaves": 31,
    "max_depth": -1,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "seed": 42
}


In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
model = lgb.train(params, lgb_train, num_boost_round=300)


In [ ]:
import pandas as pd

# Tahminleri yuvarla
preds = model.predict(X_test)
preds = np.round(preds, 2)

# Tahmin DataFrame'i oluştur
final = pd.DataFrame({
    "id": test["id"],
    "ürün fiyatı": preds
})

final = pd.DataFrame({'id': test['id'], 'ürün fiyatı': preds})

final.to_csv('submission.csv', index=False)

print("\nSubmission dosyası 'submission.csv' olarak kaydedildi.")
print(final.head())



Submission dosyası 'submission.csv' olarak kaydedildi.
   id  ürün fiyatı
0   0        88.36
1   1        29.39
2   2        31.26
3   3        18.88
4   4        34.23


In [ ]:
final = pd.DataFrame({'id': test['id'], 'ürün fiyatı': preds})

final.to_csv('submission.csv', index=False)

print("\nSubmission dosyası 'submission.csv' olarak kaydedildi.")
print(final.head())

In [ ]:
test = pd.read_csv("testFeatures.csv")


In [ ]:
# Başlıkla birlikte yazdır
print("Tahminler virgülden sonra 2 basamakla:\n")
print(final)